In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
database = client["mydatabase"]  # Replace "mydatabase" with your database name
collection = database["students"]  # Replace "students" with your collection name

# Create a new document
document = {
    "name": "John Doe",
    "age": 20,
    "grade": "A"
}

# Insert the document into the collection
collection.insert_one(document)

# Print success message
print("Document inserted successfully.")



from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra import ConsistencyLevel

def insert_product(product):
    # Connect to Cassandra
    auth_provider = PlainTextAuthProvider(username='your_username', password='your_password')
    cluster = Cluster(['localhost'], auth_provider=auth_provider)
    session = cluster.connect('your_keyspace')  # Replace 'your_keyspace' with your keyspace name

    # Create a new record
    query = """
        INSERT INTO products (id, name, price)
        VALUES (%s, %s, %s)
    """
    parameters = (product['id'], product['name'], product['price'])

    # Execute the insert query with parameters
    prepared = session.prepare(query)
    statement = prepared.bind(parameters)
    statement.consistency_level = ConsistencyLevel.QUORUM  # Set desired consistency level

    try:
        session.execute(statement)
        print("Record inserted successfully.")
    except Exception as e:
        print(f"Error inserting record: {e}")
    finally:
        session.shutdown()
        cluster.shutdown()

# Example usage
product = {
    "id": 1,
    "name": "Product 1",
    "price": 9.99
}

insert_product(product)


In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta

def fetch_recent_books():
    # Connect to MongoDB
    client = MongoClient("mongodb://localhost:27017/")
    database = client["mydatabase"]  # Replace "mydatabase" with your database name
    collection = database["books"]  # Replace "books" with your collection name

    # Calculate the date one year ago
    one_year_ago = datetime.now() - timedelta(days=365)

    # Fetch books published in the last year
    query = {"publication_date": {"$gt": one_year_ago}}
    projection = {"title": 1, "author": 1, "_id": 0}
    books = collection.find(query, projection)

    # Print titles and authors of recent books
    for book in books:
        print(f"Title: {book['title']}, Author: {book['author']}")

# Call the function
fetch_recent_books()




from pymongo import MongoClient

def insert_customer(customer):
    # Connect to MongoDB
    client = MongoClient("mongodb://localhost:27017/")
    database = client["mydatabase"]  # Replace "mydatabase" with your database name
    collection = database["customers"]  # Replace "customers" with your collection name

    # Validate the customer document
    if "name" not in customer:
        print("Customer name is required.")
        return

    if "email" not in customer:
        print("Customer email is required.")
        return

    # Insert the customer document
    collection.insert_one(customer)
    print("Customer inserted successfully.")

# Example usage
new_customer = {
    "name": "John Doe",
    "email": "johndoe@example.com",
    "address": "123 Main St",
    "phone": "555-1234"
}

insert_customer(new_customer)


In [ ]:
from pymongo import MongoClient

# Connect to MongoDB replica set
client = MongoClient("mongodb://host1:port1,host2:port2,host3:port3/?replicaSet=myReplicaSet")

# Retrieve the status of primary and secondary nodes
status = client.admin.command("replSetGetStatus")
members = status["members"]

# Print the status of each member
for member in members:
    if member["stateStr"] == "PRIMARY":
        print(f"Primary Node: {member['name']}")
    elif member["stateStr"] == "SECONDARY":
        print(f"Secondary Node: {member['name']}")



from cassandra.cluster import Cluster

# Connect to Cassandra cluster
cluster = Cluster(['host1', 'host2', 'host3'])  # Replace with actual hostnames

# Fetch the status of nodes
session = cluster.connect()
rows = session.execute("SELECT * FROM system.local")

# Print the status of each node
for row in rows:
    print(f"Node: {row.broadcast_address}, Status: {row.release_version}")

# Close the connection
session.shutdown()
cluster.shutdown()



In [ ]:
from pymongo import MongoClient

# Connect to the MongoDB cluster
client = MongoClient("mongodb://mongos1:27017,mongos2:27017/?replicaSet=myReplicaSet")

# Enable sharding for the database
admin_db = client.admin
admin_db.command({"enableSharding": "mydatabase"})

# Shard the collection on a specific field
admin_db.command({
    "shardCollection": "mydatabase.mycollection",
    "key": {"shardKeyField": 1}
})

# Insert documents into the sharded collection
db = client.mydatabase
collection = db.mycollection

documents = [
    {"shardKeyField": 1, "data": "Document 1"},
    {"shardKeyField": 2, "data": "Document 2"},
    {"shardKeyField": 3, "data": "Document 3"}
]

collection.insert_many(documents)

# Print success message
print("Documents inserted successfully.")


from pymongo import MongoClient

# Connect to the MongoDB cluster
client = MongoClient("mongodb://mongos1:27017,mongos2:27017/?replicaSet=myReplicaSet")

# Enable sharding for the database
admin_db = client.admin
admin_db.command({"enableSharding": "social_media_db"})

# Shard the collection based on user ID
admin_db.command({
    "shardCollection": "social_media_db.users",
    "key": {"user_id": 1}
})

# Insert user data into the sharded collection
db = client.social_media_db
collection = db.users

users = [
    {"user_id": 1, "name": "User 1"},
    {"user_id": 2, "name": "User 2"},
    {"user_id": 3, "name": "User 3"}
]

collection.insert_many(users)

# Fetch user data from the sharded collection
user_id = 2
user_data = collection.find_one({"user_id": user_id})

# Print the retrieved user data
print(f"User ID: {user_data['user_id']}, Name: {user_data['name']}")


In [ ]:
from pymongo import MongoClient
from pymongo import ASCENDING

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
database = client["mydatabase"]  # Replace "mydatabase" with your database name
collection = database["mycollection"]  # Replace "mycollection" with your collection name

# Create an index on the "name" field
collection.create_index([("name", ASCENDING)])

# Execute a query using the index
query = {"name": "product_name"}
result = collection.find(query)

# Print the query results
for doc in result:
    print(doc)



from pymongo import MongoClient
from pymongo import TEXT

def search_products(keyword):
    # Connect to MongoDB
    client = MongoClient("mongodb://localhost:27017/")
    database = client["mydatabase"]  # Replace "mydatabase" with your database name
    collection = database["products"]  # Replace "products" with your collection name

    # Create indexes on "name" and "description" fields
    collection.create_index([("name", TEXT)])
    collection.create_index([("description", TEXT)])

    # Execute the search query
    query = {"$text": {"$search": keyword}}
    result = collection.find(query)

    # Print the search results
    for doc in result:
        print(doc)

# Example usage
search_products("keyword")
